### Init ray

### Load default config

In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)
# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/Active_v0.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])
    
    
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

indices = list(range(len(x_train)))
random.seed(101)
random.shuffle(indices)
labeled_set = indices[:config["RUNS"]["ADDENDUM"] ]
unlabeled_set = indices[config["RUNS"]["ADDENDUM"] :]

config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

2020-12-15 14:19:13,085	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,

In [2]:
config

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0},
 'DATASET': {'height_shift_range': 4,
  'horizontal_flip': True,
  'width_shift_range': 4},
 'NETWORK': {'CLASSES': ['plane',
   'car',
   'bird',
   'cat',
   'deer',
   'dog',
   'frog',
   'horse',
   'ship',
   'truck'],
  'INPUT_SIZE': 32,
  'MARGIN': 1.0,
  'embedding_size': 128},
 'PROJECT': {'Backbone': 'resnet18',
  'dataset_name': 'CIFAR',
  'group': 'Active_Learning_keras_v0',
  'group_dir': '/mnt/Ressources/Andres/Temp_active/runs/Active_Learning_keras_v0',
  'project': 'Active_Learning_CIFAR',
  'project_dir': '/mnt/Ressources/Andres/Temp_active/runs',
  'source': 'CIFAR'},
 'RUNS': {'ADDENDUM': 1000,
  'CYCLES': 10,
  'SUBSET': -1,
  'TRIALS': 1,
  'test_each': 5},
 'TEST': {'batch_size': 128},
 'TRAIN': {'Data_augementation': True,
  'EPOCH_SLIT': 80,
  'EPOCH_WARMUP': 2,
  'EPOCH_WHOLE': 120,
  'MILESTONES': [160],
  'batch_size': 128,
  'gamma': 0.1,
  'lr': 0.001,
  'start_epoch': 0,
  'transfer_weight_path': False,
  'w_c_loss':

In [3]:
from train_agent_cifar_keras import Active_Learning_train


num_run =0
initial_weight_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run-1),'checkpoint','epoch200.ckpt-200')
initial_weight_path = False

for num_run in range(10):
    prev_num_run = num_run-1
    if num_run==0:
        resume_model_path = False
    else:
        resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run-1),'checkpoints','checkpoint.200.hdf5')
        
        
    NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  num_run, resume_model_path, resume=False)
    NetworkActor.start_training.remote()
    
    # Wait util the model is training
    while True:
        time.sleep(10)
        try:
            progress_id = NetworkActor.isTraining.remote()
            response = ray.get(progress_id)
            break
        except:
            pass
        
    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = NetworkActor.isTraining.remote()
        response = ray.get(progress_id)
        if not response:
            break
    
    NetworkActor.__ray_terminate__.remote()
    
    del NetworkActor

    num_images = (num_run+2)*config["RUNS"]["ADDENDUM"]
    labeled_set = indices[: num_images]
    unlabeled_set = indices[num_images :]

(pid=15791) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15791)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=15791) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15791)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=15791) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=15791)   _np_qint16 = np.dtyp

(pid=15791) Train_Stage_0 Init done
(pid=15791) Train_Stage_0 Start training


(pid=15791) 2020-12-15 14:19:27,250	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
(pid=15791) 2020-12-15 14:19:27,251	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.
(pid=15791) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=15791) Instructions for updating:
(pid=15791) Use tf.cast instead.
(pid=2312, ip=192.168.8.55) 2020-12-15 14:19:32.069738: W tensorflow/core/framework/op_kernel.cc:1401] OP_REQUIRES failed at tensor_array_ops.cc:447 : Invalid argument: TensorArray map/TensorArray_1_202: Could not write to TensorArray index 67 because the value shape is [92479,2] which is incompatible with the TensorArray's inferred element shape: [132300,2] (consider setting infer_shape=False).
(pid=2312, ip=192.168.8.55) 2020-12-15 14:19:3

(pid=15791) Change to split learning
(pid=15791) Previus weigths 1.0 0
(pid=15791) Updated weigths 0 1.0


(pid=15791) 
(pid=15791) wandb: Waiting for W&B process to finish, PID 19085
(pid=15791) wandb: Program ended successfully.
(pid=6695, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=6695, ip=192.168.8.54)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=6695, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=6695, ip=192.168.8.54)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=6695, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

...

 97% 97/100 [00:18<00:00,  7.57it/s]) 
 98% 98/100 [00:18<00:00,  7.90it/s]) 


(pid=19683, ip=192.168.8.53) Cache disk full, cleaning.
(pid=19683, ip=192.168.8.53) Cache disk full, cleaning.


 99% 99/100 [00:18<00:00,  8.42it/s]) 
100% 100/100 [00:18<00:00,  8.32it/s] 


(pid=19683, ip=192.168.8.53) Cache disk full, cleaning.


100% 100/100 [00:21<00:00,  4.65it/s] 
(pid=19683, ip=192.168.8.53) 
(pid=19683, ip=192.168.8.53) wandb: Waiting for W&B process to finish, PID 12527
(pid=19683, ip=192.168.8.53) wandb: Program ended successfully.
(pid=19683, ip=192.168.8.53) wandb: Run summary:
(pid=19683, ip=192.168.8.53) wandb:               _step 117698
(pid=19683, ip=192.168.8.53) wandb:            _runtime 1100.9819815158844
(pid=19683, ip=192.168.8.53) wandb:          _timestamp 1608048768.0554519
(pid=19683, ip=192.168.8.53) wandb:     mAP all classes 54.86707755241858
(pid=19683, ip=192.168.8.53) wandb: Syncing files in ../../../Trainings/wandb/wandb/run-20201215_161218-2wfg5hig:
(pid=19683, ip=192.168.8.53) wandb:   code/default_worker.py
(pid=19683, ip=192.168.8.53) wandb: plus 8 W&B file(s) and 6 media file(s)
(pid=19683, ip=192.168.8.53) wandb: - 0.01MB of 0.01MB uploaded
(pid=19683, ip=192.168.8.53) wandb: \ 0.95MB of 0.95MB uploaded
(pid=19683, ip=192.168.8.53) wandb: | 0.95MB of 0.95MB uploaded
(pid=196